# Commandes

In [ ]:
# cd ~/twinews-logs ; jupython --venv st-venv ~/notebooks/twinews/indexing.ipynb

# Init

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *

In [ ]:
import sys ; sys.path.append(homeDir() + '/Workspace/Python/Datasets/Twinews')
from twinews.utils import *

In [5]:
logger = Logger(tmpDir('logs') + "/twinews-indexing.log") if isNotebook else Logger("twinews-indexing.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [6]:
TEST = False
initialDatasetVersion = 1 if isNotebook else 3
datasetVersion = "1.0"

In [7]:
dataRootPath = dataDir() + "/Twinews/" + "twinews" + str(initialDatasetVersion)

In [8]:
newsFiles = sortedGlob(dataRootPath + "/news/*.bz2")
bp(newsFiles, logger)
bp(list(NDJson(random.choice(newsFiles)))[0].keys(), 5, logger)
bp(list(NDJson(random.choice(newsFiles)))[0]['scrap'].keys(), 5, logger)
if TEST:
    newsFiles = newsFiles[:10]

[
  /NoSave/Data/Twinews/twinews1/news/9news.com.bz2,
  /NoSave/Data/Twinews/twinews1/news/10news.com.bz2,
  ...,
  /NoSave/Data/Twinews/twinews1/news/zdnet.com.bz2,
  /NoSave/Data/Twinews/twinews1/news/zerohedge.com.bz2
]
dict_keys(['domain', 'lastUrlDomain', 'scrap', 'url', 'title', 'redirected', 'lastUrl'])
dict_keys(['meta_data', 'publish_date', 'meta_description', 'authors', 'title', 'meta_lang', 'text', 'tags', 'meta_favicon'])


In [9]:
usersFiles = sortedGlob(dataRootPath + "/users/*.bz2")
bp(usersFiles, logger)
bp(list(NDJson(random.choice(usersFiles)))[0].keys(), 5, logger)

[ /NoSave/Data/Twinews/twinews1/users/part0.bz2, /NoSave/Data/Twinews/twinews1/users/part1.bz2 ]
dict_keys(['tweet_count', 'location', 'bio', 'tweets', 'list_count', 'joindate_title', 'joindate_text', 'moment_count', 'notBotScore', 'verified', 'username', 'user_website', 'follower_count', 'following_count', 'favorite_count', 'user_id', 'datasetRelevanceScore', 'url', 'name', 'media_count', 'tweets_en_ratio', 'consecutive_old_tweets', 'avatar', 'news', 'user_twitter_website', 'date_limit'])


In [10]:
(hjuser, hjpass, hjhost) = getMongoAuth(user='hayj')

In [11]:
newsCollection = getNewsCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)
usersCollection = getUsersCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)

twinews news (version 1.0) initialised.
twinews users (version 1.0) initialised.


In [12]:
# !!!!!! DELETION OF THE DATABASE !!!!!!
newsCollection.resetCollection(security=False)
usersCollection.resetCollection(security=False)

twinews news (version 1.0) initialised.


In [13]:
assert len(newsCollection) == 0
assert len(usersCollection) == 0

In [14]:
tt.tic("Init done")

--> tic: 4.35s | message: Init done


4.35

# News

In [15]:
def preprocessNews(row, logger=None, verbose=True):
    try:
        if isinstance(row, str):
            text = row
        else:
            if dictContains(row, "scrap"):
                row = row['scrap']
            if dictContains(row, "text"):
                text = row['text']
            else:
                raise Exception("No text found in " + b(row))
        text = newsPreclean(text)
        isGood = isGoodArticle(text)
        rawText = text
        (text, tokens) = newsParser.parseNews(rawText, logger=logger, verbose=verbose)
        return (rawText, text, tokens, isGood)
    except Exception as e:
        logException(e, logger, verbose=verbose)

In [16]:
def newsGenFunct(containers, logger=None, verbose=True):
    if not isinstance(containers, list):
        containers = [containers]
    for container in containers:
        for row in NDJson(container, logger=logger, verbose=verbose):
            
            current = dictSelect(row, {'domain', 'lastUrlDomain', 'url', 'title', 'redirected', 'lastUrl'})
            scrap = row['scrap']
            current["title"] = preprocess(current["title"], doRemoveUrls=True, unescapeHtml=True,
                                  removeHtml=True, doQuoteNormalization=True,
                                  doReduceBlank=True, keepNewLines=False, logger=logger)
            (rawText, text, sentences, isGood) = preprocessNews(scrap, logger=logger)
            current["rawText"] = rawText
            current["text"] = text
            current["sentences"] = sentences
            # current["isGoodArticle"] = isGood
            if isGood and text is not None and rawText is not None and sentences is not None and len(sentences) > 0:
                yield current
            else:
                yield None

In [17]:
newsFilesChunks = chunks(newsFiles, int(len(shuffle(newsFiles)) / (cpuCount() * 16)))
bp(newsFilesChunks, logger)

[
  [
    /NoSave/Data/Twinews/twinews1/news/9news.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/10news.com.bz2,
    ...,
    /NoSave/Data/Twinews/twinews1/news/972mag.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/1843magazine.com.bz2
  ],
  [
    /NoSave/Data/Twinews/twinews1/news/abajournal.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/abc.net.au.bz2,
    ...,
    /NoSave/Data/Twinews/twinews1/news/abc15.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/abcactionnews.com.bz2
  ],
  ...,
  [
    /NoSave/Data/Twinews/twinews1/news/yallpolitics.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/yanpritzker.com.bz2,
    ...,
    /NoSave/Data/Twinews/twinews1/news/yorkdispatch.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/yorkshireeveningpost.co.uk.bz2
  ],
  [
    /NoSave/Data/Twinews/twinews1/news/zdnet.com.bz2,
    /NoSave/Data/Twinews/twinews1/news/zerohedge.com.bz2
  ]
]


In [18]:
# We add all news (22 mins for the v1 dataset):
mli = MLIterator(newsFilesChunks, newsGenFunct, logger=logger, parallelProcesses=cpuCount(), printRatio=0.001)
notGoodCount = 0
totalCount = 0
duplicatesCount = 0
hashes = set()
for row in mli:
    if row is None:
        notGoodCount += 1
    else:
        h = objectToHash(row["text"])
        if h in hashes:
            duplicatesCount += 1
        hashes.add(h)
        newsCollection.insert(row)
    totalCount += 1
log("We removed " + str(int(notGoodCount / totalCount * 100)) + "% of news.", logger)
log("Count of duplicates: " + str(duplicatesCount), logger)
log("% of duplicates: " + str(duplicatesCount / totalCount * 100), logger)

With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
198 containers to process.
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / f

...

 90% [==================  ] (41.055s left)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


 90% [==================  ] (38.752s left)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loading the GoogArticle model wich is a <class 'sklearn.svm._classes.LinearSVC'>


 91% [==================  ] (36.438s left)
 91% [==================  ] (34.152s left)
 92% [==================  ] (31.963s left)
 92% [==================  ] (30.002s left)
 93% [==================  ] (27.713s left)
 93% [==================  ] (25.607s left)
 94% [==================  ] (23.455s left)
 94% [==================  ] (21.345s left)
 95% [=================== ] (19.189s left)
 95% [=================== ] (17.196s left)
 96% [=================== ] (15.73s left)
 96% [=================== ] (13.455s left)
 97% [=================== ] (11.267s left)
 97% [=================== ] (8.985s left)
 98% [=================== ] (6.886s left)
 98% [=================== ] (4.63s left)
 99% [=================== ] (2.976s left)
100% [====================] (total duration: 22m 30.46s, mean duration: 6.82s)
We removed 8% of news.
Count of duplicates: 1750
% of duplicates: 7.117582462276813


In [19]:
# We construct a strucutre: hash of the text --> url
newsToKeepByHash = dict()
for _id, row in newsCollection.items():
    h = objectToHash(row["text"])
    newsToKeepByHash[h] = row['url']
newsToKeep = set(newsToKeepByHash.values())
bp(newsToKeepByHash, logger)
assert len(newsToKeep) == len(newsToKeepByHash)
# We construct a structure: the duplicate --> the reference news to take into account
duplicates = dict()
for _id, row in newsCollection.items():
    h = objectToHash(row["text"])
    if newsToKeepByHash[h] != row['url']:
        duplicates[row['url']] = newsToKeepByHash[h]
bp(duplicates, logger)
# We remove duplicates in news:
for dup in duplicates:
    newsCollection.delete({"url": dup})

{
  0006db2155479807dc59d944f54658c3: https://goo.gl/dLfKa5,
  000ea98707fbca374fc16e95def75b95: http://www.bbc.co.uk/news/technology-41928847,
  00115d6f3a322ea9da6cbcd867fe99b1: http://bit.ly/2lSWvxl,
  00162b5b60bcafcb823ba2e780563760: https://www.armytimes.com/off-duty/gearscout/shot-show/2018/01/22/the-rattler-this-ultra-compact-rif,
  00178dc81d34dd31a93004313cbbd333: https://www.rawstory.com/2018/02/minnesota-candidate-gets-anti-muslim-death-threats-militia-movement,
  ...,
  ffedc81ff74d09aa4cbbd88e7efc9c69: https://www.thespec.com/news-story/8100970-court-hears-dead-girl-had-drugs-in-her-system/#.WnElzoryq,
  fff4c675fb5bd94506dfd830430983f4: https://www.nytimes.com/2017/11/17/upshot/income-inequality-united-states.html,
  fffa3f193b0546ec03a7fed794f51cee: https://www.theguardian.com/commentisfree/2017/dec/01/republican-tax-bill-immoral-violence?CMP=Share,
  fffcf8893da78fecee03b51912eb9316: https://www.vice.com/en_us/article/7x9x9z/a-bartender-and-her-favorite-regular-went-so

In [20]:
tt.tic("News indexed")

--> tic: 22m 48.91s | message: News indexed


1368.91

# Users

In [21]:
def usersGenFunct(containers, maxUsersPerContainer=None, logger=None, verbose=True):
    if not isinstance(containers, list):
        containers = [containers]
    for container in containers:
        usersCount = 0
        for row in NDJson(container, logger=logger, verbose=verbose):
            try:
                assert dictContains(row, "tweets")
                # We handle news:
                currentNews = dict()
                for tweet in row["tweets"]:
                    for n in tweet['news']:
                        currentNews[n] = tweet['timestamp']
                newCurrentNews = dict()
                for n, ts in currentNews.items():
                    if n in duplicates:
                        n = duplicates[n]
                    if n in newsToKeep:
                        newCurrentNews[n] = ts
                newCurrentNews = sortBy(newCurrentNews, index=1)
                row["news"] = [e[0] for e in newCurrentNews]
                row["timestamps"] = [e[1] for e in newCurrentNews]
                # We parse tweets:
                for tweet in row["tweets"]:
                    if dictContains(tweet, "text"):
                        tweet["text"] = preprocess\
                        (
                            tweet["text"], logger=logger,
                            doQuoteNormalization=True,
                            doReduceBlank=True,
                            keepNewLines=True,
                            stripAccents=True,
                            doRemoveUrls=True,
                            doLower=False,
                            doBadlyEncoded=True,
                            doReduceCharSequences=True,
                            charSequencesMaxLength=3,
                            replaceUnknownChars=True,
                            unknownReplacer=" ",
                            doSpecialMap=True,
                            doNormalizeEmojis=True,
                            doTokenizingHelp=True,
                        )
                yield row
                usersCount += 1
                if maxUsersPerContainer is not None and usersCount >= maxUsersPerContainer:
                    break
            except Exception as e:
                logException(e, logger)

In [22]:
mli = MLIterator\
(
    usersFiles, usersGenFunct,
    genKwargs={"maxUsersPerContainer": 30 if TEST else None},
    logger=logger, parallelProcesses=cpuCount(),
    printRatio=0.001,
)
for row in mli:
    usersCollection.insert(row)

With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
2 containers to process.
 50% [==========          ] (5m 49.829s left)
100% [====================] (total duration: 6m 11.68s, mean duration: 3m 5.84s)
--> tic: 6m 11.73s | message: Users indexed


371.73

In [ ]:
tt.tic("Users indexed")

# Adding users references in news

In [45]:
# We iterate all users to make the newsUsersMapping:
newsUsersMapping = dict()
ids = usersCollection.distinct("user_id")
for userId in pb(ids, logger=logger, message="Finding all users by news"):
    data = usersCollection.findOne({"user_id": userId})
    for i in range(len(data["news"])):
        news = data["news"][i]
        ts = data["timestamps"][i]
        if news not in newsUsersMapping:
            newsUsersMapping[news] = dict()
        newsUsersMapping[news][userId] = ts
bp(newsUsersMapping, logger)
tt.tic("Users by news collected")

Finding all users by news   0% [                    ]
Finding all users by news  10% [==                  ] (9.27s left)
Finding all users by news  20% [====                ] (7.88s left)
Finding all users by news  30% [======              ] (6.906s left)
Finding all users by news  40% [========            ] (6.105s left)
Finding all users by news  50% [==========          ] (5.01s left)
Finding all users by news  60% [============        ] (4s left)
Finding all users by news  70% [==============      ] (3.025s left)
Finding all users by news  80% [================    ] (2s left)
Finding all users by news  90% [==================  ] (0.995s left)
Finding all users by news 100% [====================] (total duration: 9.92s, mean duration: 0.019s)
{
  http://2020.washingtonexaminer.com/2017/11/14/welcome-trump-county-usa/: { 252877167: 1510784442 },
  http://a.msn.com/00/en-us/AAuRcJD?ocid=st: { 265004300: 1516371177 },
  http://a.msn.com/00/en-us/AAuyKsY?ocid=st: { 17000424: 1515699392 

3454.57

In [48]:
countOfGreaterThan1 = 0
total = 0
for url, users in newsUsersMapping.items():
    if len(users) > 1:
        # log(url + " --> " + b(users), logger)
        countOfGreaterThan1 += 1
    total += len(users)

In [47]:
log("total: " + str(total), logger)
log("countOfGreaterThan1: " + str(countOfGreaterThan1), logger)
log("Mean users amount per news: " + str(total / len(newsUsersMapping)), logger)

total: 23878
countOfGreaterThan1: 1878
Mean users amount per news: 1.1613248382860757


In [49]:
# We insert the newsUsersMapping:
for url, users in pb(newsUsersMapping.items(), logger=logger, message="Inserting user list in all news rows"):
    users = sortBy(users, index=1)
    newsCollection.updateOne({"url": url},
    {
        "$set":
        {
            "users": [e[0] for e in users],
            "timestamps": [e[1] for e in users]
        }
    })
tt.tic("newsUsersMapping inserted")

Inserting user list in all news rows   0% [                    ]
Inserting user list in all news rows   9% [=                   ] (2m 17.887s left)
Inserting user list in all news rows  19% [===                 ] (1m 24.525s left)
Inserting user list in all news rows  29% [=====               ] (1m 3.097s left)
Inserting user list in all news rows  39% [=======             ] (49.474s left)
Inserting user list in all news rows  49% [=========           ] (47.844s left)
Inserting user list in all news rows  59% [===========         ] (35.744s left)
Inserting user list in all news rows  69% [=============       ] (25.349s left)
Inserting user list in all news rows  79% [===============     ] (23.995s left)
Inserting user list in all news rows  89% [=================   ] (11.313s left)
Inserting user list in all news rows  99% [=================== ] (0.005s left)
Inserting user list in all news rows 100% [====================] (total duration: 1m 47.519s, mean duration: 0.005s)
--> tic: 4m

291.18

# Deleting news having no users

In [37]:
# Searching news that has no users:
toDeleteNews = []
for _id, row in newsCollection.items():
    if not dictContains(row, 'users') or len(row['users']) == 0:
        toDeleteNews.append(row['url'])
bp(toDeleteNews, logger)
log(str(len(toDeleteNews) / len(newsCollection) * 100) + " % of news to delete because no user shared it.", logger)

[
  http://wwd.com/business-news/media/conde-nast-to-close-teen-vogue-cut-80-jobs-and-lower-mag-frequenc,
  https://www.yahoo.com/amphtml/music/cyntonia-brown-life-sentence-sparks-141514557.html,
  ...,
  https://fb.me/1s6zYkAzE,
  https://tinyurl.com/ybxb5zm4
]
852.3809523809524 % of news to delete because no user shared it.


In [40]:
# Deleting news:
for url in toDeleteNews:
    newsCollection.delete({'url': url})

In [ ]:
tt.tic("News having no users deleted")

# End

In [ ]:
tt.toc()